# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.nn import functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm
from tqdm import trange
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [ ]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
class SimpleNet(nn.Module):
    # TODO:define model
    # initialize the model
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 6, 5, padding = 2),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d((2,2))
        )
        
        self.connect = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )
        
    def forward(self, x):
        y = self.layer(x)
        y = y.view(-1, self.num_flat_features(y))
        y = self.connect(y)
        return y
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for key in size:
            num_features *= key
        return num_features

    
model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=.1, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Next, we can start to train and evaluate!

In [ ]:
# train and evaluate
loss_list = []
train_auc_list = []
auc_list = []
def train(model, optimizer, criterion, train_loader, device, epoch, loss_list, train_auc_list):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    for idx, (images, labels) in enumerate(train_loader):
        # TODO:forward + backward + optimize      
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
        total += labels.size(0)

    print('Epoch {} training loss {}, training accuracy {}'.format(epoch,running_loss/len(train_loader), 100*correct/total))
    loss_list.append(running_loss/len(train_loader))
    train_auc_list.append(100*correct/total)
        

        
def test(model, criterion, test_loader, device, epoch, auc_list):
    model.eval()
    correct = 0
    total = 0
    cnt = 0
    with torch.no_grad():
        for idx, (images, labels) in enumerate(test_loader):
            output = model(images)
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(labels.data.view_as(pred)).cpu().sum()
            total += labels.size(0)

    print('Epoch {} testing accuracy:{}'.format(epoch, 100*correct/total))
    auc_list.append(100*correct/total)
    
    
for epoch in trange(0, NUM_EPOCHS):
    train(model, optimizer, criterion, train_loader, device, epoch, loss_list, train_auc_list)
    test(model, criterion, test_loader, device, epoch, auc_list)

#### Q5:
Please print the training and testing accuracy.

In [ ]:
for i in range(NUM_EPOCHS):
    print('Epoch {} training accuracy {}, testing accuracy {}'.format(i+1, train_auc_list[i], auc_list[i]))

'''
Epoch 1 training accuracy 98.87653350830078, testing accuracy 98.26722717285156
Epoch 2 training accuracy 98.82478332519531, testing accuracy 98.69791412353516
Epoch 3 training accuracy 99.09688568115234, testing accuracy 98.56771087646484
Epoch 4 training accuracy 99.03345489501953, testing accuracy 98.41746520996094
Epoch 5 training accuracy 98.9967269897461, testing accuracy 98.0869369506836
Epoch 6 training accuracy 98.96501159667969, testing accuracy 98.46754455566406
Epoch 7 training accuracy 99.07351684570312, testing accuracy 98.0068130493164
Epoch 8 training accuracy 99.07184600830078, testing accuracy 98.12700653076172
Epoch 9 training accuracy 98.92327880859375, testing accuracy 97.96674346923828
Epoch 10 training accuracy 98.90324401855469, testing accuracy 98.36738586425781
'''